In [6]:
public record struct Point {
    public int X;
    public int Y;
    public int Z;

    public static Point operator+(Point position, Point other) {
        return new Point {
            X = position.X + other.X,
            Y = position.Y + other.Y,
            Z = position.Z + other.Z,
        };
    }
    
    public static Point operator-(Point position, Point other) {
        return new Point {
            X = position.X - other.X,
            Y = position.Y - other.Y,
            Z = position.Z - other.Z,
        };
    }
}

public record struct Bounds {
    public int MinX;
    public int MinY;
    public int MinZ;
    public int MaxX;
    public int MaxY;
    public int MaxZ;

    public static Bounds From(IEnumerable<Point> points) {
        return new Bounds {
            MinX = points.Min(p => p.X),
            MaxX = points.Max(p => p.X),
            MinY = points.Min(p => p.Y),
            MaxY = points.Max(p => p.Y),
            MinZ = points.Min(p => p.Z),
            MaxZ = points.Max(p => p.Z),
        };
    }

    public static Bounds Grow(Bounds bounds, int size) {
        return new Bounds {
            MinX = bounds.MinX - size,
            MaxX = bounds.MaxX + size,
            MinY = bounds.MinY - size,
            MaxY = bounds.MaxY + size,
            MinZ = bounds.MinZ - size,
            MaxZ = bounds.MaxZ + size,
        };
    }

    public bool ContainsPoint(Point point) {
        return point.X >= MinX
            && point.X <= MaxX
            && point.Y >= MinY
            && point.Y <= MaxY
            && point.Z >= MinZ
            && point.Z <= MaxZ;
    }
}

In [7]:
var input = System.IO.File.ReadAllLines("input.txt");
input.Take(10)

index,value
0,"18,15,10"
1,"12,7,16"
2,"14,2,8"
3,"14,14,18"
4,"5,7,4"
5,"4,11,4"
6,"10,17,6"
7,"18,10,9"
8,"18,11,13"
9,"15,4,15"


In [8]:
var allPoints = input.Select(line => line.Split(","))
    .Select(values => new Point { X = int.Parse(values[0]), Y = int.Parse(values[1]), Z = int.Parse(values[2]) } )
    .ToHashSet();

var deltas = new List<Point> {
    new Point { X = 1, Y = 0, Z = 0 },
    new Point { X = -1, Y = 0, Z = 0 },
    new Point { X = 0, Y = 1, Z = 0 },
    new Point { X = 0, Y = -1, Z = 0 },
    new Point { X = 0, Y = 0, Z = 1 },
    new Point { X = 0, Y = 0, Z = -1 },
};

allPoints.Select(point =>
    deltas.Select(delta => point + delta)
        .Where(neighbour => allPoints.Contains(neighbour) == false)
        .Count()
    )
    .Sum()


4608

In [9]:
var rockPoints = input.Select(line => line.Split(","))
    .Select(values => new Point { X = int.Parse(values[0]), Y = int.Parse(values[1]), Z = int.Parse(values[2]) } )
    .ToHashSet();

var deltas = new List<Point> {
    new Point { X = 1, Y = 0, Z = 0 },
    new Point { X = -1, Y = 0, Z = 0 },
    new Point { X = 0, Y = 1, Z = 0 },
    new Point { X = 0, Y = -1, Z = 0 },
    new Point { X = 0, Y = 0, Z = 1 },
    new Point { X = 0, Y = 0, Z = -1 },
};

var bounds = Bounds.Grow(Bounds.From(rockPoints), 1);
var startingPoint = new Point { X = bounds.MinX, Y = bounds.MinY, Z = bounds.MinZ };

var pointsToExplore = new Stack<Point>(new List<Point> { startingPoint });
var seenPoints = new HashSet<Point>();
var surfaceAreaCount = 0;

while (pointsToExplore.TryPop(out var nextPoint)) {
    if (seenPoints.Contains(nextPoint)) {
        continue;
    }
    
    var neighbours = deltas.Select(delta => nextPoint + delta).ToList();
    surfaceAreaCount += neighbours.Where(neighbour => rockPoints.Contains(neighbour))
        .Count();
    var unseenSteamPoints = neighbours.Where(bounds.ContainsPoint)
        .Where(neighbour => seenPoints.Contains(neighbour) == false)
        .Where(neighbour => rockPoints.Contains(neighbour) == false);
    foreach (var point in unseenSteamPoints) {
        pointsToExplore.Push(point);
    }

    seenPoints.Add(nextPoint);
}
surfaceAreaCount

2652